# Setup

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [2]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r")


 ········


In [60]:
from langchain_community.utilities import SQLDatabase

# Provide the correct path to the Chinook database
db_path = "C:/Users/scb/Chinook.db"

# Create SQLDatabase instance with the correct URI
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")


sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [22]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

llm = ChatCohere(model="command-r")

 ········


# Chains

In [37]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'Question: How many employees are there?\nSQLQuery: ```sql\nSELECT COUNT(*) FROM Employee;\n```\nSQLResult:'

In [38]:
import re
# Extract the SQL query using a regular expression
sql_query_match = re.search(r"```sql\n(.*?)\n```", response, re.DOTALL)
if sql_query_match:
    sql_query = sql_query_match.group(1).strip()
else:
    raise ValueError("SQL query not found in the response.")


In [39]:
db.run(sql_query)

'[(8,)]'

In [40]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [49]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'Error: (sqlite3.OperationalError) near "Question": syntax error\n[SQL: Question: How many employees are there?\nSQLQuery: ```sql\nSELECT COUNT(*) FROM Employee;\n```\nSQLResult:]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [50]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many employees are there"})

'There is an error in executing the SQL query. The error message is: *(sqlite3.OperationalError) near "Question": syntax error*. It seems like the SQL query is not properly formed.'

In [61]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000282E071D400>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000282E071D400>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000282E071D400>),
 QuerySQLCheckerTool(description='Use this tool to 

In [62]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [63]:
%%capture --no-stderr
%pip install --upgrade --quiet langgraph

In [64]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

In [65]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Which country's customers spent the most?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '37cbd927-f3c9-4434-b83f-16a8037bd63e', 'tool_calls': [{'id': 'f741d82d76414924bca3cd05ecfc4ac1', 'function': {'name': 'sql_db_list_tables', 'arguments': '{}'}, 'type': 'function'}, {'id': 'b480920fb195416292165eeae300c11e', 'function': {'name': 'sql_db_schema', 'arguments': '{"table_names": "Customers, Orders"}'}, 'type': 'function'}, {'id': 'dc5220dbc83348378bba61bcb8911cbe', 'function': {'name': 'sql_db_query_checker', 'arguments': '{"query": "SELECT Country, SUM(Amount) AS Total_Spent\\nFROM Customers C JOIN Orders O ON C.Customer_ID = O.Customer_ID\\nGROUP BY Country\\nORDER BY Total_Spent DESC LIMIT 1"}'}, 'type': 'function'}, {'id': '26b9981b07324352898ee0d70ff21865', 'function': {'name': 'sql_db_query', 'arguments': '{"query": "SELECT Country, SUM(Amount) AS Total_Spent\\nFROM Customer

In [66]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Describe the playlisttrack table")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1e5500f7-e610-46f9-9b60-a55907a204bd', 'tool_calls': [{'id': '9d2e013128cf49ffa0e455de53116264', 'function': {'name': 'sql_db_list_tables', 'arguments': '{}'}, 'type': 'function'}, {'id': '8e7c1cdaade4450f9527f7eca57b9f44', 'function': {'name': 'sql_db_schema', 'arguments': '{"table_names": "playlisttrack"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 21}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1e5500f7-e610-46f9-9b60-a55907a204bd', 'tool_calls': [{'id': '9d2e013128cf49ffa0e455de53116264', 'function': {'name': 'sql_db_list_tables', 'arguments': '{}'}, 'type': 'function'}, {'id': '8e7c1cdaade4450f9527f7eca57b9f44', 'function': {'name': 'sql_db_schema', 'arguments': '{"t

In [70]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")
albums[:5]

['Blizzard of Ozz',
 'Jagged Little Pill',
 'Surfing with the Alien (Remastered)',
 'Roda De Funk',
 'O Samba Poconé']

In [76]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings

vector_db = FAISS.from_texts(artists + albums, CohereEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [77]:
print(retriever_tool.invoke("Alice Chains"))

Alice In Chains

AC/DC

Pearl Jam

Pearl Jam

Iron Maiden


In [78]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

agent = create_react_agent(llm, tools, messages_modifier=system_message)

In [79]:
for s in agent.stream(
    {"messages": [HumanMessage(content="How many albums does alis in chain have?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '2c79c29c-7abb-4dd8-b486-504d61c5102e', 'tool_calls': [{'id': '0220d82115eb4e929008320485f27436', 'function': {'name': 'sql_db_schema', 'arguments': '{"table_names": "Artist, Album"}'}, 'type': 'function'}, {'id': '421af218810a4427b396237ca9b4aac0', 'function': {'name': 'sql_db_query_checker', 'arguments': '{"query": "SELECT COUNT(Album.AlbumId) FROM Artist JOIN Album ON Artist.ArtistId  =  Album.ArtistId WHERE Artist.Name  =  \'Alis in Chains\'"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 62}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '2c79c29c-7abb-4dd8-b486-504d61c5102e', 'tool_calls': [{'id': '0220d82115eb4e929008320485f27436', 'function': {'name': 'sql_db_schema', 'a